# Prompt Formatter Tutorial

This tutorial introduces NeMo's PromptFormatter API available in module `nemo.collections.common.prompts`.
After finishing this tutorial you will be familiar with the existing prompt formatters, how to use them, and how to build your own.

We cover the following topics:

* Using existing prompt formatters with Llama2 as an example.

* Defining your own prompt formatter.

We also support applying prompt formatters for multimodal data and Lhotse-compatible data types. To learn more, see our other tutorial: [Multimodal Lhotse Dataloading](./Multimodal Lhotse Dataloading.ipynb)

### Pre-requsite: building a dummy tokenizer

We're going to need a tokenizer to work with prompt formatters - we'll just build a dummy one for the purpose of this tutorial.

In [1]:
import string
import shlex
from nemo.collections.common.tokenizers.sentencepiece_tokenizer import SentencePieceTokenizer, create_spt_model

!echo {shlex.quote(' '.join(string.printable))} > _tutorial_train_text.txt

tok_path, vocab_path = create_spt_model(
    data_file="_tutorial_train_text.txt", 
    output_dir="_tutorial_spt",
    vocab_size=512, 
    sample_size=-1, 
    do_lower_case=False, 
    bos=True, 
    eos=True, 
    pad=True, 
    user_defined_symbols=["[INST]", "[/INST]", "<<SYS>>", "<</SYS>>", "[audio]"]
)

tokenizer = SentencePieceTokenizer(tok_path)

def display(encoded_chat, with_mask=False):
    """Utility for printing prompt formatted chats."""
    for key, val in encoded_chat.items():
        if key.endswith("_ids"):
            print(key, '--', tokenizer.ids_to_text(val), '\n')
        if key == "mask" and with_mask:
            print(key, '--', val)

/Users/pzelasko/miniforge3/envs/nemo/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


[NeMo I 2024-10-23 11:26:41 sentencepiece_tokenizer:333] tokenizer model _tutorial_spt/tokenizer.model already exists


## Using an existing PromptFormatter: Llama2


**Instanting the prompt formatter.** Let's start with a simple example of Llama2 prompt format use.

In [2]:
from nemo.collections.common.prompts.llama import Llama2PromptFormatter
from pprint import pprint

prompt = Llama2PromptFormatter(tokenizer)

**Chat example.** We'll define a multi-turn conversation between the user and assistant below:

In [3]:
chat = [
    {"role": "user", "slots": {"message": "Do you know something about electronics?"}},
    {"role": "assistant", "slots": {"message": "Sure, ask away."}},
    {"role": "user", "slots": {"message": "How to build my own audio amplifier?"}},
    {"role": "assistant", "slots": {"message": "In order to build your own audio amplifier, start with ..."}},
]

**Prompt formatter outputs.** Now, we apply prompt formatter to that conversation to obtain four tensors useful for training:
* `context_ids` encode the whole dialog history up to the last response of the assistant;
* `answer_ids` encode the last response of the assistant;
* `input_ids` encode the full conversation;
* `mask` is a boolean training loss mask that's set to `True` for every token belonging to assistant's turns.

Since the token IDs are meaningless, we'll apply reverse tokenizer for displaying the prompt formatted example.

In [4]:
encoded = prompt.encode_dialog(chat)
display(encoded, with_mask=True)

input_ids -- [INST] Do you know something about electronics? [/INST] Sure, ask away. [INST] How to build my own audio amplifier? [/INST] In order to build your own audio amplifier, start with ... 

context_ids -- [INST] Do you know something about electronics? [/INST] Sure, ask away. [INST] How to build my own audio amplifier? [/INST] 

answer_ids -- In order to build your own audio amplifier, start with ... 

mask -- tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True, False, False, False, False, False, False, False,
        False, False, False, Fal

**System prompt.** We also support the system prompt. Since it affects the prompt format in a non-trivial way, it is defined as a separate role `"system_and_user"`, which has two slots `"system"` and `"message"`. We'll omit printing the mask for brevity.

In [5]:
chat_with_system = [
    {"role": "system_and_user", "slots": {"system": "You are a sales rep in an electronics store.", "message": "Do you know something about electronics?"}},
    {"role": "assistant", "slots": {"message": "Sure, ask away."}},
    {"role": "user", "slots": {"message": "How to build my own audio amplifier?"}},
    {"role": "assistant", "slots": {"message": "In order to build your own audio amplifier, start with ..."}},
]

In [6]:
encoded = prompt.encode_dialog(chat_with_system)
display(encoded)

input_ids -- [INST] <<SYS>> You are a sales rep in an electronics store. <</SYS>> Do you know something about electronics? [/INST] Sure, ask away. [INST] How to build my own audio amplifier? [/INST] In order to build your own audio amplifier, start with ... 

context_ids -- [INST] <<SYS>> You are a sales rep in an electronics store. <</SYS>> Do you know something about electronics? [/INST] Sure, ask away. [INST] How to build my own audio amplifier? [/INST] 

answer_ids -- In order to build your own audio amplifier, start with ... 



**Constructing inference-time prompts.** During inference, we don't know what's the last turn of the assistant - we only want to construct the ``context_ids`` tensor. In those cases, just omit the last assistant's turn. The prompt formatter will return the ``context_ids`` tensor (with ``input_ids`` alias for it too).

In [7]:
inference_chat = [
    {"role": "system_and_user", "slots": {"system": "You are a sales rep in an electronics store.", "message": "Do you know something about electronics?"}},
    {"role": "assistant", "slots": {"message": "Sure, ask away."}},
    {"role": "user", "slots": {"message": "How to build my own audio amplifier?"}},
]

In [8]:
encoded = prompt.encode_dialog(inference_chat)
display(encoded)

input_ids -- [INST] <<SYS>> You are a sales rep in an electronics store. <</SYS>> Do you know something about electronics? [/INST] Sure, ask away. [INST] How to build my own audio amplifier? [/INST] 

context_ids -- [INST] <<SYS>> You are a sales rep in an electronics store. <</SYS>> Do you know something about electronics? [/INST] Sure, ask away. [INST] How to build my own audio amplifier? [/INST] 



### How is Llama2 PromptFormatter built

`Llama2PromptFormatter` is a small class with prompt definition that inherits `PromptFormatter`, which implements the logic for applying prompt format and tokenization to multi-turn conversations. 

Let's take a look at `Llama2PromptFormatter` definition:

In [9]:
import inspect
print(inspect.getsource(Llama2PromptFormatter))

class Llama2PromptFormatter(PromptFormatter):
    """
    This template has been validated to provide identical tokenized results to the official code
    in https://github.com/meta-llama/llama/blob/main/llama/generation.py
    """

    NAME = "llama2"
    OUTPUT_ROLE = "assistant"
    TEMPLATE = {
        "system_and_user": {
            "template": f"{BOS_SLOT}[INST] <<SYS>>\n|system|\n<</SYS>>\n\n|message| [/INST]",
            "slots": {
                "system": Modality.Text,
                "message": Modality.Text,
            },
        },
        "user": {
            "template": f"{BOS_SLOT}[INST] |message| [/INST]",
            "slots": {
                "message": Modality.Text,
            },
        },
        OUTPUT_ROLE: {
            "template": f"|message| {EOS_SLOT}",
            "slots": {
                "message": Modality.Text,
            },
        },
    }



As you can see, the definition consist of the following key components:
* Derives `PromptFormatter` parent class.
* Specifies `NAME`, which is used for dynamic resolution of string to class via `cls = PromptFormatter.resolve(name)`.
* Specifies `OUTPUT_ROLE`, which is the name for the role with assistant's responses (typically `"assistant"`).
* Specifies `TEMPLATE` which defines the dialog structure and how user-provided values (slots) are applied to prompts. Notably:
  * The slots are wrapped into pipe operators `"|"` in the prompt template definition, and substituted with user provided values before tokenization.
  * `"system_and_user`" role has two slots, `"system"` and `"message"`, and a template that wraps them with Llama2 special tokens.
  * We use `BOS_SLOT` and `EOS_SLOT` to insert sentencepiece tokenizer's `bos_id` and `eos_id` in the right places (remember that sentencepiece won't tokenize them from text, they need to be inserted programmatically).
  * The slots have a type, currently supported types are `Modality.Text` and `Modality.TextLiteral(value1, value2, ...)` that allows to restrict the set of slots values.

## Defining your own prompt formatter

Generally you can follow the definition of existing prompt formatters to define your own. 
We have several prompt formats implemented for Llama, Gemma, Phi, etc. 

We'll define a custom simple prompt format that has no system prompt below as an illustration:

In [10]:
from nemo.collections.common.prompts.formatter import PromptFormatter, Modality

class MyPrompt(PromptFormatter):
    NAME = "myprompt"
    OUTPUT_ROLE = "assistant"
    TEMPLATE = {
        "user": {
            "template": "User: |message|\n",
            "slots": {"message": Modality.Text},
        },
        "assistant": {
            "template": "Assistant: |message|\n",
            "slots": {"message": Modality.Text},
        },
    }

In [11]:
my_prompt_cls = PromptFormatter.resolve("myprompt")  # it is auto-registered
my_prompt = my_prompt_cls(tokenizer)
display(my_prompt.encode_dialog(chat))

input_ids -- User: Do you know something about electronics? Assistant: Sure, ask away. User: How to build my own audio amplifier? Assistant: In order to build your own audio amplifier, start with ... 

context_ids -- User: Do you know something about electronics? Assistant: Sure, ask away. User: How to build my own audio amplifier? 

answer_ids -- Assistant: In order to build your own audio amplifier, start with ... 



## Applying prompt formatter to multimodal data

We refer the reader to our other tutorial, [Multimodal Lhotse Dataloading](./Multimodal Lhotse Dataloading.ipynb), where this is discussed in detail.